## TFMA Notebook example

This notebook describes how to export your model for TFMA and demonstrates the analysis tooling it offers.

## Setup

Import necessary packages.

In [1]:
import apache_beam as beam  
import os
import preprocess
import shutil
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
from google.protobuf import text_format 
from tensorflow.python.lib.io import file_io
from tensorflow_transform.beam.tft_beam_io import transform_fn_io
from tensorflow_transform.coders import example_proto_coder
from tensorflow_transform.saved import saved_transform_io
from tensorflow_transform.tf_metadata import dataset_schema
from tensorflow_transform.tf_metadata import schema_utils
from trainer import task
from trainer import taxi

Helper functions and some constants for running the notebook locally.

In [2]:
BASE_DIR = os.getcwd()

DATA_DIR = os.path.join(BASE_DIR, 'data')

OUTPUT_DIR = os.path.join(BASE_DIR, 'chicago_taxi_output')

# Base dir containing train and eval data
TRAIN_DATA_DIR = os.path.join(DATA_DIR, 'train')
EVAL_DATA_DIR = os.path.join(DATA_DIR, 'eval')

# Base dir where TFT writes training data
TFT_TRAIN_OUTPUT_BASE_DIR = os.path.join(OUTPUT_DIR, 'tft_train')
TFT_TRAIN_FILE_PREFIX = 'train_transformed'

# Base dir where TFT writes eval data
TFT_EVAL_OUTPUT_BASE_DIR = os.path.join(OUTPUT_DIR, 'tft_eval')
TFT_EVAL_FILE_PREFIX = 'eval_transformed'

TF_OUTPUT_BASE_DIR = os.path.join(OUTPUT_DIR, 'tf')

# Base dir where TFMA writes eval data
TFMA_OUTPUT_BASE_DIR = os.path.join(OUTPUT_DIR, 'tfma')

SERVING_MODEL_DIR = 'serving_model_dir'
EVAL_MODEL_DIR = 'eval_model_dir'


def get_tft_train_output_dir(run_id):
    return _get_output_dir(TFT_TRAIN_OUTPUT_BASE_DIR, run_id)


def get_tft_eval_output_dir(run_id):
    return _get_output_dir(TFT_EVAL_OUTPUT_BASE_DIR, run_id)


def get_tf_output_dir(run_id):
    return _get_output_dir(TF_OUTPUT_BASE_DIR, run_id)

def get_tfma_output_dir(run_id):
    return _get_output_dir(TFMA_OUTPUT_BASE_DIR, run_id)

def _get_output_dir(base_dir, run_id):
    return os.path.join(base_dir, 'run_' + str(run_id))

def get_schema_file():
    return os.path.join(OUTPUT_DIR, 'schema.pbtxt')


Clean up output directories.

In [3]:
shutil.rmtree(TFT_TRAIN_OUTPUT_BASE_DIR, ignore_errors=True)
shutil.rmtree(TFT_EVAL_OUTPUT_BASE_DIR, ignore_errors=True)
shutil.rmtree(TF_OUTPUT_BASE_DIR, ignore_errors=True)
shutil.rmtree(get_schema_file(), ignore_errors=True)

## Compute and visualize descriptive data statistics

In [4]:
# Compute stats over training data.
train_stats = tfdv.generate_statistics_from_csv(data_location=os.path.join(TRAIN_DATA_DIR, 'data.csv'))

In [5]:
# Visualize training data stats.
tfdv.visualize_statistics(train_stats)

## Infer a schema

In [6]:
# Infer a schema from the training data stats.
schema = tfdv.infer_schema(statistics=train_stats, infer_feature_shape=False)
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'fare',FLOAT,required,single,-
'trip_start_hour',INT,required,single,-
'pickup_census_tract',BYTES,optional,,-
'dropoff_census_tract',FLOAT,optional,single,-
'company',STRING,optional,single,'company'
'trip_start_timestamp',INT,required,single,-
'pickup_longitude',FLOAT,required,single,-
'trip_start_month',INT,required,single,-
'trip_miles',FLOAT,required,single,-


,Values
Domain,
'company',"'0118 - 42111 Godfrey S.Awir', '0694 - 59280 Chinesco Trans Inc', '1085 - 72312 N and W Cab Co', '2733 - 74600 Benny Jona', '2809 - 95474 C & D Cab Co Inc.', '3011 - 66308 JBL Cab Inc.', '3152 - 97284 Crystal Abernathy', '3201 - C&D Cab Co Inc', '3201 - CID Cab Co Inc', '3253 - 91138 Gaither Cab Co.', '3385 - 23210 Eman Cab', '3623 - 72222 Arrington Enterprises', '3897 - Ilie Malec', '4053 - Adwar H. Nikola', '4197 - 41842 Royal Star', '4615 - 83503 Tyrone Henderson', '4615 - Tyrone Henderson', '4623 - Jay Kim', '5006 - 39261 Salifu Bawa', '5006 - Salifu Bawa', '5074 - 54002 Ahzmi Inc', '5074 - Ahzmi Inc', '5129 - 87128', '5129 - 98755 Mengisti Taxi', '5129 - Mengisti Taxi', '5724 - KYVI Cab Inc', '585 - Valley Cab Co', '5864 - 73614 Thomas Owusu', '5864 - Thomas Owusu', '5874 - 73628 Sergey Cab Corp.', '5997 - 65283 AW Services Inc.', '5997 - AW Services Inc.', '6488 - 83287 Zuha Taxi', '6743 - Luhak Corp', 'Blue Ribbon Taxi Association Inc.', 'C & D Cab Co Inc', 'Chicago Elite Cab Corp.', 'Chicago Elite Cab Corp. (Chicago Carriag', 'Chicago Medallion Leasing INC', 'Chicago Medallion Management', 'Choice Taxi Association', 'Dispatch Taxi Affiliation', 'KOAM Taxi Association', 'Northwest Management LLC', 'Taxi Affiliation Services', 'Top Cab Affiliation'"
'payment_type',"'Cash', 'Credit Card', 'Dispute', 'No Charge', 'Pcard', 'Unknown'"


## Check evaluation data for errors

In [7]:
# Compute stats over eval data.
eval_stats = tfdv.generate_statistics_from_csv(data_location=os.path.join(EVAL_DATA_DIR, 'data.csv'))

In [8]:
# Compare stats of eval data with training data.
tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats,
                          lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

In [9]:
# Check eval data for errors by validating the eval data stats using the previously inferred schema.
anomalies = tfdv.validate_statistics(statistics=eval_stats, schema=schema)
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'company',Unexpected string values,"Examples contain values missing from the schema: 2092 - 61288 Sbeih company (<1%), 2192 - 73487 Zeymane Corp (<1%), 2192 - Zeymane Corp (<1%), 2823 - 73307 Seung Lee (<1%), 3094 - 24059 G.L.B. Cab Co (<1%), 3319 - CD Cab Co (<1%), 3385 - Eman Cab (<1%), 3897 - 57856 Ilie Malec (<1%), 4053 - 40193 Adwar H. Nikola (<1%), 4197 - Royal Star (<1%), 585 - 88805 Valley Cab Co (<1%), 5874 - Sergey Cab Corp. (<1%), 6057 - 24657 Richard Addo (<1%), 6574 - Babylon Express Inc. (<1%), 6742 - 83735 Tasha ride inc (<1%)."
'payment_type',Unexpected string values,Examples contain values missing from the schema: Prcard (<1%).


In [10]:
# Update the schema based on the observed anomalies.

# Relax the minimum fraction of values that must come from the domain for feature company.
company = tfdv.get_feature(schema, 'company')
company.distribution_constraints.min_domain_mass = 0.9

# Add new value to the domain of feature payment_type.
payment_type_domain = tfdv.get_domain(schema, 'payment_type')
payment_type_domain.value.append('Prcard')

# Validate eval stats after updating the schema 
updated_anomalies = tfdv.validate_statistics(eval_stats, schema)
tfdv.display_anomalies(updated_anomalies)

## Freeze the schema

Now that the schema has been reviewed and curated, we will store it in a file to reflect its "frozen" state.

In [11]:
file_io.recursive_create_dir(OUTPUT_DIR)
file_io.write_string_to_file(get_schema_file(), text_format.MessageToString(schema))

## Preprocess Inputs

transform_data is defined in preprocess.py and uses the tensorflow_transform library to perform preprocessing. The same code is used for both local preprocessing in this notebook and preprocessing in the Cloud (via Dataflow).

In [12]:
# Transform eval data
preprocess.transform_data(input_handle=os.path.join(EVAL_DATA_DIR, 'data.csv'),
                          outfile_prefix=TFT_EVAL_FILE_PREFIX, 
                          working_dir=get_tft_eval_output_dir(0),
                          schema_file=get_schema_file(),
                          pipeline_args=['--runner=DirectRunner'])
print('Done')

input shiape ## SparseTensor(indices=Tensor("inputs/company/indices_copy:0", shape=(?, 2), dtype=int64), values=Tensor("inputs/company/values_copy:0", shape=(?,), dtype=string), dense_shape=Tensor("inputs/company/shape_copy:0", shape=(2,), dtype=int64))
INFO:tensorflow:Assets added to graph.


INFO:tensorflow:Assets added to graph.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: /Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tft_eval/run_0/tftransform_tmp/b700c44a9eeb4b97a7f6db7fcdaf115c/saved_model.pb


...

value: "\n\013\n\tConst_9:0\022/vocab_compute_and_apply_vocabulary_1_vocabulary"



value: "\n\013\n\tConst_9:0\022/vocab_compute_and_apply_vocabulary_1_vocabulary"



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Done


In [13]:
# Transform training data
preprocess.transform_data(input_handle=os.path.join(TRAIN_DATA_DIR, 'data.csv'),
                          outfile_prefix=TFT_TRAIN_FILE_PREFIX, 
                          working_dir=get_tft_train_output_dir(0),
                          schema_file=get_schema_file(),
                          pipeline_args=['--runner=DirectRunner'])
print('Done')

input shiape ## SparseTensor(indices=Tensor("inputs/company/indices_copy:0", shape=(?, 2), dtype=int64), values=Tensor("inputs/company/values_copy:0", shape=(?,), dtype=string), dense_shape=Tensor("inputs/company/shape_copy:0", shape=(2,), dtype=int64))
INFO:tensorflow:Assets added to graph.


INFO:tensorflow:Assets added to graph.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: /Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tft_train/run_0/tftransform_tmp/c00d60748a184987849bba85efbda6c4/saved_model.pb


...

value: "\n\013\n\tConst_9:0\022/vocab_compute_and_apply_vocabulary_1_vocabulary"



value: "\n\013\n\tConst_9:0\022/vocab_compute_and_apply_vocabulary_1_vocabulary"



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Done


## Compute statistics over transformed data 

In [14]:
# Compute stats over transformed training data.
TRANSFORMED_TRAIN_DATA = os.path.join(get_tft_train_output_dir(0), TFT_TRAIN_FILE_PREFIX + "*") 
transformed_train_stats = tfdv.generate_statistics_from_tfrecord(data_location=TRANSFORMED_TRAIN_DATA)

In [15]:
# Visualize transformed training data stats and compare to raw training data. 
# Use 'Feature search' to focus on a feature and see statistics pre- and post-transformation.
tfdv.visualize_statistics(transformed_train_stats, train_stats, lhs_name='TRANSFORMED', rhs_name='RAW')

## Prepare the Model

To use TFMA, export the model into an **EvalSavedModel** by calling ``tfma.export.export_eval_savedmodel``.

``tfma.export.export_eval_savedmodel`` is analogous to ``estimator.export_savedmodel`` but exports the evaluation graph as opposed to the training or inference graph. Notice that one of the inputs is ``eval_input_receiver_fn`` which is analogous to ``serving_input_receiver_fn`` for ``estimator.export_savedmodel``. For more details, refer to the documentation for TFMA on Github.

Contruct the **EvalSavedModel** after training is completed.

In [16]:
def run_experiment(hparams):
    """Run the training and evaluate using the high level API"""

    # Train and evaluate the model as usual.
    estimator = task.train_and_maybe_evaluate(hparams)

    # Export TFMA's sepcial EvalSavedModel
    eval_model_dir = os.path.join(hparams.output_dir, EVAL_MODEL_DIR)
    receiver_fn = lambda: eval_input_receiver_fn(hparams.tf_transform_dir)

    tfma.export.export_eval_savedmodel(
        estimator=estimator,
        export_dir_base=eval_model_dir,
        eval_input_receiver_fn=receiver_fn)
    
def eval_input_receiver_fn(working_dir):
    # Extract feature spec from the schema.
    raw_feature_spec = schema_utils.schema_as_feature_spec(schema).feature_spec

    serialized_tf_example = tf.placeholder(
        dtype=tf.string, shape=[None], name='input_example_tensor')

    # First we deserialize our examples using the raw schema.
    features = tf.parse_example(serialized_tf_example, raw_feature_spec)

    # Now that we have our raw examples, we must process them through tft
    _, transformed_features = (
        saved_transform_io.partially_apply_saved_transform(
            os.path.join(working_dir, transform_fn_io.TRANSFORM_FN_DIR),
            features))

    # The key MUST be 'examples'.
    receiver_tensors = {'examples': serialized_tf_example}
    
    # NOTE: Model is driven by transformed features (since training works on the
    # materialized output of TFT, but slicing will happen on raw features.
    features.update(transformed_features)
    
    return tfma.export.EvalInputReceiver(
        features=features,
        receiver_tensors=receiver_tensors,
        labels=transformed_features[taxi.transformed_name(taxi.LABEL_KEY)])

print('Done')

Done


## Train and export the model for TFMA

In [17]:
def run_local_experiment(tft_run_id, tf_run_id, num_layers, first_layer_size, scale_factor):
    """Helper method to train and export the model for TFMA
    
    The caller specifies the input and output directory by providing run ids. The optional parameters
    allows the user to change the modelfor time series view.
    
    Args:
      tft_run_id: The run id for the preprocessing. Identifies the folder containing training data.
      tf_run_id: The run for this training run. Identify where the exported model will be written to.
      num_layers: The number of layers used by the hiden layer.
      first_layer_size: The size of the first hidden layer.
      scale_factor: The scale factor between each layer in in hidden layers.
    """
    hparams = tf.contrib.training.HParams(
        # Inputs: are tf-transformed materialized features
        train_files=os.path.join(get_tft_train_output_dir(tft_run_id), TFT_TRAIN_FILE_PREFIX + '-00000-of-*'),
        eval_files=os.path.join(get_tft_eval_output_dir(tft_run_id), TFT_EVAL_FILE_PREFIX + '-00000-of-*'),
        schema_file=get_schema_file(),
        # Output: dir for trained model
        job_dir=get_tf_output_dir(tf_run_id),
        tf_transform_dir=get_tft_train_output_dir(tft_run_id),
        
        # Output: dir for both the serving model and eval_model which will go into tfma
        # evaluation
        output_dir=get_tf_output_dir(tf_run_id),
        train_steps=10000,
        eval_steps=5000,
        num_layers=num_layers,
        first_layer_size=first_layer_size,
        scale_factor=scale_factor,
        num_epochs=None,
        train_batch_size=40,
        eval_batch_size=40)

    run_experiment(hparams)

print('Done')

Done


In [18]:
run_local_experiment(tft_run_id=0,
                     tf_run_id=0,
                     num_layers=4,
                     first_layer_size=100,
                     scale_factor=0.7)
print('Done')

INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_num_ps_replicas': 0, '_keep_checkpoint_max': 1, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x120f74310>, '_model_dir': '/Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_0/serving_model_dir', '_protocol': None, '_save_checkpoints_steps': 999, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}


INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_num_ps_replicas': 0, '_keep_checkpoint_max': 1, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x120f74310>, '_model_dir': '/Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_0/serving_model_dir', '_protocol': None, '_save_checkpoints_steps': 999, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}


INFO:tensorflow:Running training and evaluation locally (non-distributed).


INFO:tensorflow:Running training and evaluation locally (non-distributed).


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 999 or save_checkpoints_secs None.


...

INFO:tensorflow:Assets written to: /Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_0/eval_model_dir/temp-1550347248/assets


INFO:tensorflow:Assets written to: /Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_0/eval_model_dir/temp-1550347248/assets


INFO:tensorflow:SavedModel written to: /Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_0/eval_model_dir/temp-1550347248/saved_model.pb


INFO:tensorflow:SavedModel written to: /Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_0/eval_model_dir/temp-1550347248/saved_model.pb


Done


## Run TFMA to compute metrics
For local analysis, TFMA offers a helper method ``tfma.run_model_analysis``

In [19]:
help(tfma.run_model_analysis)

Help on function run_model_analysis in module tensorflow_model_analysis.api.model_eval_lib:

run_model_analysis(eval_shared_model, data_location, file_format='tfrecords', slice_spec=None, output_path=None, extractors=None)
    Runs TensorFlow model analysis.
    
    It runs a Beam pipeline to compute the slicing metrics exported in TensorFlow
    Eval SavedModel and returns the results.
    
    This is a simplified API for users who want to quickly get something running
    locally. Users who wish to create their own Beam pipelines can use the
    Evaluate PTransform instead.
    
    Args:
      eval_shared_model: Shared model parameters for EvalSavedModel including any
        additional metrics (see EvalSharedModel for more information on how to
        configure additional metrics).
      data_location: The location of the data files.
      file_format: The file format of the data, can be either 'text' or
        'tfrecords' for now. By default, 'tfrecords' will be used.
      sl

#### You can also write your own custom pipeline if you want to perform extra transformations on the data before evaluation.

In [20]:
def run_tfma(slice_spec, tf_run_id, tfma_run_id, input_csv, schema_file, add_metrics_callbacks=None):
    """A simple wrapper function that runs tfma locally.
    
    A function that does extra transformations on the data and then run model analysis.
    
    Args:
        slice_spec: The slicing spec for how to slice the data.
        tf_run_id: An id to contruct the model directories with.
        tfma_run_id: An id to construct output directories with.
        input_csv: The evaluation data in csv format.
        schema_file: The file holding a text-serialized schema for the input data.
        add_metrics_callback: Optional list of callbacks for computing extra metrics.
        
    Returns:
        An EvalResult that can be used with TFMA visualization functions.
    """
    eval_model_base_dir = os.path.join(get_tf_output_dir(tf_run_id), EVAL_MODEL_DIR)
    eval_model_dir = os.path.join(eval_model_base_dir, next(os.walk(eval_model_base_dir))[1][0])
    eval_shared_model = tfma.default_eval_shared_model(
        eval_saved_model_path=eval_model_dir,
        add_metrics_callbacks=add_metrics_callbacks)
    schema = taxi.read_schema(schema_file)
    
    print(eval_model_dir)
    
    display_only_data_location = input_csv
    
    with beam.Pipeline() as pipeline:
        csv_coder = taxi.make_csv_coder(schema)
        raw_data = (
            pipeline
            | 'ReadFromText' >> beam.io.ReadFromText(
                input_csv,
                coder=beam.coders.BytesCoder(),
                skip_header_lines=True)
            | 'ParseCSV' >> beam.Map(csv_coder.decode))
        
        # Examples must be in clean tf-example format.
        coder = taxi.make_proto_coder(schema)
        raw_data = (
            raw_data
            | 'ToSerializedTFExample' >> beam.Map(coder.encode))

        _ = (raw_data
             | 'ExtractEvaluateAndWriteResults' >>
             tfma.ExtractEvaluateAndWriteResults(
                 eval_shared_model=eval_shared_model,
                 slice_spec=slice_spec,
                 output_path=get_tfma_output_dir(tfma_run_id),
                 display_only_data_location=input_csv))

    return tfma.load_eval_result(output_path=get_tfma_output_dir(tfma_run_id))
    
print('Done')

Done


#### You can also compute metrics on slices of your data in TFMA. Slices can be specified using ``tfma.slicer.SingleSliceSpec``.

Below are examples of how slices can be specified.

In [21]:
# An empty slice spec means the overall slice, that is, the whole dataset.
OVERALL_SLICE_SPEC = tfma.slicer.slicer.SingleSliceSpec()

# Data can be sliced along a feature column
# In this case, data is sliced along feature column trip_start_hour.
FEATURE_COLUMN_SLICE_SPEC = tfma.slicer.slicer.SingleSliceSpec(columns=['trip_start_hour'])

# Data can be sliced by crossing feature columns
# In this case, slices are computed for trip_start_day x trip_start_month.
FEATURE_COLUMN_CROSS_SPEC = tfma.slicer.slicer.SingleSliceSpec(columns=['trip_start_day', 'trip_start_month'])

# Metrics can be computed for a particular feature value.
# In this case, metrics is computed for all data where trip_start_hour is 12.
FEATURE_VALUE_SPEC = tfma.slicer.slicer.SingleSliceSpec(features=[('trip_start_hour', 12)])

# It is also possible to mix column cross and feature value cross.
# In this case, data where trip_start_hour is 12 will be sliced by trip_start_day.
COLUMN_CROSS_VALUE_SPEC = tfma.slicer.slicer.SingleSliceSpec(columns=['trip_start_day'], features=[('trip_start_hour', 12)])

ALL_SPECS = [
    OVERALL_SLICE_SPEC,
    FEATURE_COLUMN_SLICE_SPEC, 
    FEATURE_COLUMN_CROSS_SPEC, 
    FEATURE_VALUE_SPEC, 
    COLUMN_CROSS_VALUE_SPEC    
]

#### Let's run TFMA!

In [22]:
tf.logging.set_verbosity(tf.logging.INFO)

tfma_result_1 = run_tfma(input_csv=os.path.join(EVAL_DATA_DIR, 'data.csv'), 
                         tf_run_id=0, 
                         tfma_run_id=1,
                         slice_spec=ALL_SPECS,
                         schema_file=get_schema_file())
print('Done')


/Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_0/eval_model_dir/1550347248
INFO:tensorflow:Restoring parameters from /Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_0/eval_model_dir/1550347248/variables/variables


INFO:tensorflow:Restoring parameters from /Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_0/eval_model_dir/1550347248/variables/variables


Done


## Visualization: Slicing Metrics

To see the slices, either use the name of the column (by setting slicing_column) or provide a tfma.slicer.SingleSliceSpec (by setting slicing_spec). If neither is provided, the overall will be displayed.

The default visualization is **slice overview** when the number of slices is small. It shows the value of a metric for each slice sorted by the another metric. It is also possible to set a threshold to filter out slices with smaller weights.

This view also supports **metrics histogram** as an alternative visualization. It is also the defautl view when the number of slices is large. The results will be divided into buckets and the number of slices / total weights / both can be visualized. Slices with small weights can be fitlered out by setting the threshold. Further filtering can be applied by dragging the grey band. To reset the range, double click the band. Filtering can be used to remove outliers in the visualization and the metrics table below.

In [23]:
# Show data sliced along feature column trip_start_hour.
tfma.view.render_slicing_metrics(
    tfma_result_1, slicing_column='trip_start_hour')

U2xpY2luZ01ldHJpY3NWaWV3ZXIoY29uZmlnPXsnd2VpZ2h0ZWRFeGFtcGxlc0NvbHVtbic6ICdwb3N0X2V4cG9ydF9tZXRyaWNzL2V4YW1wbGVfY291bnQnfSwgZGF0YT1beydtZXRyaWNzJzrigKY=


In [24]:
# Show metrics sliced by COLUMN_CROSS_VALUE_SPEC above.
tfma.view.render_slicing_metrics(tfma_result_1, slicing_spec=COLUMN_CROSS_VALUE_SPEC)

U2xpY2luZ01ldHJpY3NWaWV3ZXIoY29uZmlnPXsnd2VpZ2h0ZWRFeGFtcGxlc0NvbHVtbic6ICdwb3N0X2V4cG9ydF9tZXRyaWNzL2V4YW1wbGVfY291bnQnfSwgZGF0YT1beydtZXRyaWNzJzrigKY=


In [25]:
# Show overall metrics.
tfma.view.render_slicing_metrics(tfma_result_1)

U2xpY2luZ01ldHJpY3NWaWV3ZXIoY29uZmlnPXsnd2VpZ2h0ZWRFeGFtcGxlc0NvbHVtbic6ICdwb3N0X2V4cG9ydF9tZXRyaWNzL2V4YW1wbGVfY291bnQnfSwgZGF0YT1beydtZXRyaWNzJzrigKY=


## Visualization: Plots

TFMA offers a number of built-in plots. To see them, add them to ``add_metrics_callbacks``

In [26]:
tf.logging.set_verbosity(tf.logging.INFO)

tfma_vis = run_tfma(input_csv=os.path.join(EVAL_DATA_DIR, 'data.csv'), 
                    tf_run_id=0,
                    tfma_run_id='vis',
                    slice_spec=ALL_SPECS,
                    schema_file=get_schema_file(),
                    add_metrics_callbacks=[
                        # calibration_plot_and_prediction_histogram computes calibration plot and prediction
                        # distribution at different thresholds.
                        tfma.post_export_metrics.calibration_plot_and_prediction_histogram(),
                        # auc_plots enables precision-recall curve and ROC visualization at different thresholds.
                        tfma.post_export_metrics.auc_plots()
                    ])

print('Done')

/Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_0/eval_model_dir/1550347248
INFO:tensorflow:Restoring parameters from /Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_0/eval_model_dir/1550347248/variables/variables


INFO:tensorflow:Restoring parameters from /Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_0/eval_model_dir/1550347248/variables/variables


Done


Plots must be visualized for an individual slice. To specify a slice, use ``tfma.slicer.SingleSliceSpec``.

In the example below, we are using ``tfma.slicer.SingleSliceSpec(features=[('trip_start_hour', 0)])`` to specify the slice where trip_start_hour is 0.

Plots are interactive:
- Drag to pan
- Scroll to zoom
- Right click to reset the view

Simply hover over the desired data point to see more details.

In [27]:
tfma.view.render_plot(tfma_vis, tfma.slicer.slicer.SingleSliceSpec(features=[('trip_start_hour', 0)]))

UGxvdFZpZXdlcihjb25maWc9eydzbGljZU5hbWUnOiAndHJpcF9zdGFydF9ob3VyOjAnLCAnbWV0cmljS2V5cyc6IHsnYXVjUGxvdCc6IHsnZGF0YVNlcmllcyc6ICdtYXRyaWNlcycsICdtZXTigKY=


#### Custom metrics

In addition to plots, it is also possible to compute additional metrics not present at export time or custom metrics metrics using ``add_metrics_callbacks``.

All metrics in ``tf.metrics`` are supported in the callback and can be used to compose other metrics:
https://www.tensorflow.org/api_docs/python/tf/metrics

In the cells below, false negative rate is computed as an example.

In [28]:
# Defines a callback that adds FNR to the result.
def add_fnr_for_threshold(threshold):
    def _add_fnr_callback(features_dict, predictions_dict, labels_dict):
        metric_ops = {}
        prediction_tensor = tf.cast(
            predictions_dict.get(tf.contrib.learn.PredictionKey.LOGISTIC), tf.float64)
        fn_value_op, fn_update_op = tf.metrics.false_negatives_at_thresholds(tf.squeeze(labels_dict), 
                                                                             tf.squeeze(prediction_tensor), 
                                                                             [threshold])
        tp_value_op, tp_update_op = tf.metrics.true_positives_at_thresholds(tf.squeeze(labels_dict), 
                                                                            tf.squeeze(prediction_tensor), 
                                                                            [threshold])
        fnr = fn_value_op[0] / (fn_value_op[0] + tp_value_op[0])
        metric_ops['FNR@' + str(threshold)] = (fnr, tf.group(fn_update_op, tp_update_op)) 
        return metric_ops
    
    return _add_fnr_callback

In [29]:
tf.logging.set_verbosity(tf.logging.INFO)

tfma_fnr = run_tfma(input_csv=os.path.join(EVAL_DATA_DIR, 'data.csv'), 
                    tf_run_id=0,
                    tfma_run_id='fnr',
                    slice_spec=ALL_SPECS,
                    schema_file=get_schema_file(),
                    add_metrics_callbacks=[
                        # Simply add the call here.
                        add_fnr_for_threshold(0.75)
                    ])
tfma.view.render_slicing_metrics(tfma_fnr, slicing_spec=FEATURE_COLUMN_SLICE_SPEC)

/Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_0/eval_model_dir/1550347248
INFO:tensorflow:Restoring parameters from /Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_0/eval_model_dir/1550347248/variables/variables


INFO:tensorflow:Restoring parameters from /Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_0/eval_model_dir/1550347248/variables/variables


U2xpY2luZ01ldHJpY3NWaWV3ZXIoY29uZmlnPXsnd2VpZ2h0ZWRFeGFtcGxlc0NvbHVtbic6ICdwb3N0X2V4cG9ydF9tZXRyaWNzL2V4YW1wbGVfY291bnQnfSwgZGF0YT1beydtZXRyaWNzJzrigKY=


## Visualization: Time Series

It is important to track how your model is doing over time. TFMA offers two modes to show your model performs over time.

**Multiple model analysis** shows how model perfoms from one version to another. This is useful early on to see how the addition of new features, change in modeling technique, etc, affects the performance. TFMA offers a convenient method.

In [30]:
help(tfma.multiple_model_analysis)

Help on function multiple_model_analysis in module tensorflow_model_analysis.api.model_eval_lib:

multiple_model_analysis(model_locations, data_location, **kwargs)
    Run model analysis for multiple models on the same data set.
    
    Args:
      model_locations: A list of paths to the export eval saved model.
      data_location: The location of the data files.
      **kwargs: The args used for evaluation. See tfma.run_model_analysis() for
        details.
    
    Returns:
      A tfma.EvalResults containing all the evaluation results with the same order
      as model_locations.



**Multiple data analysis** shows how a model perfoms under different evaluation data set. This is useful to ensure that model performance does not degrade over time. TFMA offer a conveneient method.

In [31]:
help(tfma.multiple_data_analysis)

Help on function multiple_data_analysis in module tensorflow_model_analysis.api.model_eval_lib:

multiple_data_analysis(model_location, data_locations, **kwargs)
    Run model analysis for a single model on multiple data sets.
    
    Args:
      model_location: The location of the exported eval saved model.
      data_locations: A list of data set locations.
      **kwargs: The args used for evaluation. See tfma.run_model_analysis() for
        details.
    
    Returns:
      A tfma.EvalResults containing all the evaluation results with the same order
      as data_locations.



It is also possible to compose a time series manually.

In [32]:
# Create different models.

# Run some experiments with different hidden layer configurations.
run_local_experiment(tft_run_id=0,
                     tf_run_id=1,
                     num_layers=3,
                     first_layer_size=200,
                     scale_factor=0.7)

run_local_experiment(tft_run_id=0,
                     tf_run_id=2,
                     num_layers=4,
                     first_layer_size=240,
                     scale_factor=0.5)

print('Done')

INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_num_ps_replicas': 0, '_keep_checkpoint_max': 1, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1d488c410>, '_model_dir': '/Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_1/serving_model_dir', '_protocol': None, '_save_checkpoints_steps': 999, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}


INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_num_ps_replicas': 0, '_keep_checkpoint_max': 1, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1d488c410>, '_model_dir': '/Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_1/serving_model_dir', '_protocol': None, '_save_checkpoints_steps': 999, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}


INFO:tensorflow:Running training and evaluation locally (non-distributed).


INFO:tensorflow:Running training and evaluation locally (non-distributed).


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 999 or save_checkpoints_secs None.


...

INFO:tensorflow:Assets written to: /Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_2/eval_model_dir/temp-1550347565/assets


INFO:tensorflow:Assets written to: /Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_2/eval_model_dir/temp-1550347565/assets


INFO:tensorflow:SavedModel written to: /Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_2/eval_model_dir/temp-1550347565/saved_model.pb


INFO:tensorflow:SavedModel written to: /Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_2/eval_model_dir/temp-1550347565/saved_model.pb


Done


In [33]:
tfma_result_2 = run_tfma(input_csv=os.path.join(EVAL_DATA_DIR, 'data.csv'), 
                         tf_run_id=1, 
                         tfma_run_id=2,                         
                         slice_spec=ALL_SPECS,
                         schema_file=get_schema_file())

tfma_result_3 = run_tfma(input_csv=os.path.join(EVAL_DATA_DIR, 'data.csv'), 
                         tf_run_id=2, 
                         tfma_run_id=3,
                         slice_spec=ALL_SPECS,
                         schema_file=get_schema_file())
print('Done')

/Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_1/eval_model_dir/1550347498
INFO:tensorflow:Restoring parameters from /Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_1/eval_model_dir/1550347498/variables/variables


INFO:tensorflow:Restoring parameters from /Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_1/eval_model_dir/1550347498/variables/variables


/Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_2/eval_model_dir/1550347565
INFO:tensorflow:Restoring parameters from /Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_2/eval_model_dir/1550347565/variables/variables


INFO:tensorflow:Restoring parameters from /Users/tvykruta/Documents/src/tfx/examples/chicago_taxi/chicago_taxi_output/tf/run_2/eval_model_dir/1550347565/variables/variables


Done


Like plots, time series view must visualized for a slice too.

In the example below, we are showing the overall slice.

Select a metric to see its time series graph. Hover over each data point to get more details.

In [34]:
eval_results = tfma.make_eval_results([tfma_result_1, tfma_result_2, tfma_result_3], 
                                      tfma.constants.MODEL_CENTRIC_MODE)
tfma.view.render_time_series(eval_results, OVERALL_SLICE_SPEC)
print('done.')


done.


Serialized results can also be used to construct a time series. Thus, there is no need to re-run TFMA for models already evaluated for a long running pipeline.

In [35]:
# Visualize the results in a Time Series. In this case, we are showing the slice specified.
eval_results_from_disk = tfma.load_eval_results([get_tfma_output_dir(1), 
                                                 get_tfma_output_dir(2), 
                                                 get_tfma_output_dir(3)], 
                                                tfma.constants.MODEL_CENTRIC_MODE)
tfma.view.render_time_series(eval_results_from_disk, FEATURE_VALUE_SPEC)
print('done.')

done.
